In [23]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [8]:
import os
import string
import shutil
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec


profile=webdriver.FirefoxOptions()
profile.set_preference("browser.download.folderList",2)
profile.set_preference("browser.download.manager.showWhenStarting",False)
profile.set_preference("browser.download.dir","C:/Users/pdv/Desktop/selenium-imgs/pdfs")
profile.set_preference("browser.helperApps.neverAsk.saveToDisk","application/octet-stream")
profile.add_argument("--window-size=1920,1080")
profile.add_argument('--disable-dev-shm-usage')
profile.add_argument('--disable-gpu')
profile.add_argument('--headless')
browser=webdriver.Firefox(options=profile)
            
    
def purgedir(parent):
    for root, dirs, files in os.walk(parent):                                      
        for item in files:
            filespec = os.path.join(root, item)
            os.unlink(filespec)


def buscaxml(numero=0,fornecedor='',lojas=['0'],mes='04'):
    xmls,lista = [],[]
    cnpjs = ['05349537000185','07246178000166','08158815000105','32265622000138',
             '05349537000266','23017342000209','23017342000110']
    lojitas = [cnpjs[int(i)] for i in lojas]
    for cnpj in lojitas:
        dire = f'N://XML ENTRADA/destinadas/{cnpj}/2022-{mes}'
        if fornecedor:
            arq = [f for f in os.listdir(dire) if fornecedor in f]
        if numero:
            arq = [f for f in os.listdir(dire) if str(numero) in f]
        for i in arq:
            xmls.append({f'{cnpj}':i})
    for k in xmls: 
        for cnpj,v in k.items():
            if (numero and str(numero) in v[25:34]) or fornecedor:
                lista.append(fr'N:\XML ENTRADA\destinadas\\{cnpj}\2022-{mes}\\{v}')
#                 lista.append('\n')
                   # r'N:\XML ENTRADA\destinadas\05349537000185\2022-04\35220433019177000199550010000866381022596760.xml'
    print(lista)
    return lista
        
    
def nota(xml):
    #  XML deve ser um arquivo.xml ou um caminho absoluto
    browser.get('https://www.fsist.com.br/converter-xml-nfe-para-danfe')
    lista = [(By.ID,"arquivo"),(By.CLASS_NAME,"butenviar"),(By.ID,"msgsim"),(By.ID,"butlink")]
    wait = WebDriverWait(browser, 10)
    for x,y in lista:
        if not y == 'arquivo':
            elem = wait.until(ec.element_to_be_clickable((x,y)))
            elem.click()
        else:
            elem = browser.find_element(x,y)
            print('erro no sendkeys')
            for i in xml:
                elem.send_keys(i)

    
def extrai(id1):    
    down = 'C://Users/pdv/Downloads'
    output = f'C://Users/pdv/Desktop/selenium-imgs/pdfs/{id1}'
    remove = []
    for raiz, pastas, arquivos in os.walk(down): 
        for arquivo in arquivos:
            if arquivo.startswith('FSist'):
                print(arquivo)
                entrada = os.path.join(down, arquivo)
                saida = os.path.join(output, arquivo)
                remove.append(entrada)
                with zipfile.ZipFile(entrada, mode="r") as archive:
                    for i in archive.namelist():
                        if not i.startswith('_'):
                            print(i)
                            archive.extract(i, path=output)
    for i in remove:
        os.remove(i)
        
        
def iniciapdf(id1, fornecedor = '',numero = 0, lojas = ['0'], mes = '04'):
    print('sou um pdf')
    print('numero: ',numero,' fornecedor: ', fornecedor)
    if numero != 0 and fornecedor == '':
        lojas = ['0','1','2','3','4','5','6']
    lista = buscaxml(numero, fornecedor,lojas,mes)
    if not lista:
        return 'vazio'
    print(f'{lista} = buscaxml({numero}, {fornecedor}, {lojas}, {mes}')
    try:
        os.mkdir(f'C:/Users/pdv/Desktop/selenium-imgs/pdfs/{id1}')
    except:
        purgedir(f'C:/Users/pdv/Desktop/selenium-imgs/pdfs/{id1}')
#     for i in lista:
    nota(lista)
    extrai(id1)
    return 'ok'
    
    
if __name__ == '__main__': 
    iniciapdf(945441763,fornecedor = '13495487', numero=0, lojas = ['2'], mes = '03')


sou um pdf
numero:  0  fornecedor:  13495487
['N:\\XML ENTRADA\\destinadas\\\\08158815000105\\2022-03\\\\35220313495487000172550010030629351489478866.xml', 'N:\\XML ENTRADA\\destinadas\\\\08158815000105\\2022-03\\\\35220313495487000172550010030646771940159220.xml', 'N:\\XML ENTRADA\\destinadas\\\\08158815000105\\2022-03\\\\35220313495487000172550010030646781799629854.xml', 'N:\\XML ENTRADA\\destinadas\\\\08158815000105\\2022-03\\\\35220313495487000172550010030806511727250595.xml', 'N:\\XML ENTRADA\\destinadas\\\\08158815000105\\2022-03\\\\35220313495487000172550010030845981871558533.xml']
['N:\\XML ENTRADA\\destinadas\\\\08158815000105\\2022-03\\\\35220313495487000172550010030629351489478866.xml', 'N:\\XML ENTRADA\\destinadas\\\\08158815000105\\2022-03\\\\35220313495487000172550010030646771940159220.xml', 'N:\\XML ENTRADA\\destinadas\\\\08158815000105\\2022-03\\\\35220313495487000172550010030646781799629854.xml', 'N:\\XML ENTRADA\\destinadas\\\\08158815000105\\2022-03\\\\35220313495487